In [1]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from pathlib import Path

sys.path.append("/home/acho/Sync/KiddLab/MSM/src")
from utils.stim_tools import *

sys.path.append("/home/acho/Sync/Python/sigtools")
from sigtools.representations import *
# from sigtools.sounds import *
# from sigtools.processing import *
# from sigtools.spatialization import *

|| MSM INITIALIZATION
Setting paths...
Importing sigtools...
Setting up sounddevice...
Importing level adjustments...
Setting eligible BUG words...
Set values for tone pattern synthesis...
Initialization complete!


In [32]:
from collections import namedtuple
Condition = namedtuple("Condition",
                       "stim_type target_alt_rate masker_alt_rate target_init_angle masker_init_angle")
stim_database = pd.read_csv(STIM_DIR/"stimulus_database.csv")

EXPT_1_1_1 = [Condition("SEM", 0.5, 0.5, None, None),
              Condition("SEM", 1. , 1. , None, None),
              Condition("SEM", 2. , 2. , None, None),
              Condition("SEM", 5. , 5. , None, None)]
EXPT_1_1_2 = [Condition("SIM", 0.5, 0.5, None, None),
              Condition("SIM", 1. , 1. , None, None),
              Condition("SIM", 2. , 2. , None, None),
              Condition("SIM", 5. , 5. , None, None)]
EXPT_1_2_1 = [Condition("SEM", 0. , 0.5,   0.,  90.),
              Condition("SEM", 0. , 1. ,   0.,  90.),
              Condition("SEM", 0. , 2. ,   0.,  90.),
              Condition("SEM", 0. , 5. ,   0.,  90.)]
EXPT_1_2_2 = [Condition("SIM", 0. , 0.5,   0.,  90.),
              Condition("SIM", 0. , 1. ,   0.,  90.),
              Condition("SIM", 0. , 2. ,   0.,  90.),
              Condition("SIM", 0. , 5. ,   0.,  90.)]
EXPT_1_3_1 = [Condition("SEM", 0.5, 0. ,  90.,   0.),
              Condition("SEM", 1. , 0. ,  90.,   0.),
              Condition("SEM", 2. , 0. ,  90.,   0.),
              Condition("SEM", 5. , 0. ,  90.,   0.)]
EXPT_1_3_2 = [Condition("SIM", 0.5, 0. ,  90.,   0.),
              Condition("SIM", 1. , 0. ,  90.,   0.),
              Condition("SIM", 2. , 0. ,  90.,   0.),
              Condition("SIM", 5. , 0. ,  90.,   0.)]
EXPT_2_1_1 = [Condition("SEM", 0.5, 5. , None, None),
              Condition("SEM", 5. , 0.5, None, None)]
EXPT_2_1_2 = [Condition("SIM", 0.5, 5. , None, None),
              Condition("SIM", 5. , 0.5, None, None)]

In [108]:
def choose_stim_with_curr_cond(stim_database, all_block_list, cond, n_trials_per_block):
    from functools import partial, reduce
    inner_merge = partial(pd.merge, how="inner")
    flatten = lambda t: [item for sublist in t for item in sublist] # flattens list of lists in double loop
    
    # Choose the subset of the stimulus database that satisfies the current conditions
    conditions = []
    conditions.append(stim_database[(stim_database["stim_type"] == cond.stim_type)])
    conditions.append(stim_database[(stim_database["is_target"] == True) &
                                    (stim_database["alt_rate"] == cond.target_alt_rate)])
    conditions.append(stim_database[(stim_database["is_target"] == False) &
                                    (stim_database["alt_rate"] == cond.masker_alt_rate)])
    if cond.target_init_angle: # if initial angle is specified in conditions, it will be not None
        conditions.append(stim_database[(stim_database["is_target"] == True) &
                                        (np.abs(stim_database["init_angle"]) == cond.target_init_angle)])
    if cond.masker_init_angle:
        conditions.append(stim_database[(stim_database["is_target"] == False) &
                                        (np.abs(stim_database["init_angle"]) == cond.masker_init_angle)])
    subsets = [stim_database.loc[stim_database["stim_num"].isin(cond["stim_num"])] for cond in conditions]
    conditioned_stim_database = reduce(inner_merge, subsets)

    # Determine the available stimuli by removing already used stimuli 
    eligible_stim_num = conditioned_stim_database["stim_num"].unique()
    used_stim_num = [stim_tuple[1] for stim_tuple in flatten(all_block_list)]
    available_stim_num = list(set(eligible_stim_num) - set(used_stim_num))
    selected_stim_num = sorted(np.random.choice(available_stim_num, n_trials_per_block, replace=False))

    # Load the stimuli as sounds and insert into block
    srcs = [SoundLoader(STIM_DIR/("stim_" + str(stim_num).zfill(5) + ".wav"))
            for stim_num in selected_stim_num]
    patterns = [pattern for pattern in
                stim_database.loc[(stim_database["stim_num"].isin(selected_stim_num)) & 
                                  (stim_database["is_target"])]["pattern"]]
    stim_tuple = list(zip(srcs, selected_stim_num, patterns))
    np.random.shuffle(stim_tuple)
    return stim_tuple

In [111]:
n_blocks_per_cond = 2
n_trials_per_block = 10
curr_expt = n_blocks_per_cond*EXPT_1_1_1
np.random.shuffle(curr_expt)

all_block_list = []
for cond in curr_expt:
    curr_block_list = choose_stim_with_curr_cond(stim_database, all_block_list, cond, n_trials_per_block)
    all_block_list.append(curr_block_list)

### Expt 1
* Symmetric moving target and masker, EM and IM (0.1, 0.5, 4, 8)
* Static target, moving masker, EM and IM (0.1, 0.5, 4, 8)
* Moving target, static masker, EM and IM (0.1, 0.5, 4, 8)

### Expt 2
* Moving at different rates, EM and IM ((0.1, 4), (4, 0.1))
* Predictability???

In [ ]:
subject_ID = "AYC"
task_type = "SIM"
n_srcs = 3
conditions = ["co-located", "plus_minus_90", 0.5, 2, 4] # alt rate
stim_database = pd.read_csv(STIM_DIR/"stimulus_database.csv")

# Check that the specified conditions are in the stimulus database
validate_parameters(stim_database, task_type, n_srcs, conditions)

# Blocking parameters
randomize_within_block = True
n_blocks = 1                         # if randomized within block
n_trials_per_block_per_condition = 5 # if randomized within block
n_trials_per_block = None      # if NOT randomized within block
n_blocks_per_condition = None  # if NOT randomized within block
run_stim_order = set_stim_order(stim_database, task_type, n_srcs, conditions,
                                n_blocks,
                                n_trials_per_block_per_condition,
                                n_trials_per_block,
                                n_blocks_per_condition,
                                randomize_within_block)